In [ ]:
import os.path
import nltk
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem   import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import files
files.upload() #this will prompt you to update the json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

Saving archive (1).zip to archive (1).zip
kaggle.json


In [ ]:
# Access Google Sheets as a data source.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(creds)
# At this point, you will have a link printed in your output which will direct you to a sign-in page. Pick the relevant google account and copy the provided link. Paste it in the provided line at the output section.


In [ ]:
# Load your dataframe

import pandas as pd

wb = gc.open_by_url('https://drive.google.com/file/d/14cYzq-nAaMF0OiiPrH-03bhNXAjMtQUj/view?usp=sharing') # A URL of your workbook.
sheet1 = wb.worksheet('WELFake_Dataset') # Enter your sheet name.
original_df = sheet1.get_all_values()
df = pd.DataFrame(original_df)
df.head()

NoValidUrlKeyFound: ignored

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('WELFake_Dataset').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

SpreadsheetNotFound: ignored

In [ ]:
from google.colab import files
upload = files.upload()

KeyboardInterrupt: ignored

In [ ]:
# get_all_values gives a list of rows.
rows = upload.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
wc = WordCloud().generate(df.groupby('label')['title'].sum()[0])
plt.figure(figsize=(15,15))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

In [ ]:
wc = WordCloud().generate(df.groupby('label')['title'].sum()[1])
plt.figure(figsize=(15,15))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

Text preprocessing procedure

Remove punctuation and digits
Tokenize
Lemmatize
Remove stopwords

In [ ]:
def preprocess_text(text):
    text = ''.join([c for c in text if c not in string.punctuation and c not in string.digits])
    tokens = word_tokenize(text, 'english')
    lemmatiser = WordNetLemmatizer()
    lemmatized = [lemmatiser.lemmatize(word) for word in tokens]
    sw = stopwords.words('english')
    stopped = [word for word in lemmatized if word.lower() not in sw]
    return stopped
preprocess_text(df.loc[0, 'title'])

In [ ]:
from sklearn.model_selection import train_test_split

X = df['title']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
X_train.shape[0], X_test.shape[0]

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
X_train = X_train.fillna('')
bow_transformer = CountVectorizer(analyzer=preprocess_text).fit(X_train)
text_bow_train = bow_transformer.transform(X_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

model = RandomForestClassifier(n_estimators=300)

scores = cross_validate(model, text_bow_train, y_train, scoring=['f1', 'accuracy'], cv=5, n_jobs=-1)
pd.DataFrame(scores).describe()

Now retrain the model on the whole training set

In [ ]:
model.fit(text_bow_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

X_test = X_test.fillna('')
text_bow_test = bow_transformer.transform(X_test)

y_pred = model.predict(text_bow_test)
print(classification_report(y_test, y_pred))